In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3

In [2]:
db_path = 'C:/Users/rashi/ESM_databases/temoa/data_files/'
db_name = 'canoe_on_12d_vanilla3' + '.sqlite'

In [3]:
db_file = db_path + db_name

conn = sqlite3.connect(db_file)
query_net_capacity = "SELECT * FROM OutputNetCapacity WHERE sector = 'Transport'"
query_built_capacity = "SELECT * FROM OutputBuiltCapacity WHERE sector = 'Transport'"
query_existing_capacity = "SELECT * FROM ExistingCapacity WHERE tech like 'T_%'"

net_cap = pd.read_sql_query(query_net_capacity, conn).drop(columns=['scenario', 'region', 'sector'])
new_cap = pd.read_sql_query(query_built_capacity, conn).drop(columns=['scenario', 'region', 'sector'])
ex_cap =  pd.read_sql_query(query_existing_capacity, conn)[['tech', 'vintage', 'capacity', 'units']]

conn.close()

In [4]:
class_mapping = {
    'T_HDV_AJ': 'Air jet',
    'T_HDV_B': 'Bus',
    'T_HDV_R': 'Rail',
    'T_HDV_T': 'HD Truck',
    'T_HDV_W': 'Marine vessel',
    'T_MDV_T': 'MD Truck',
    'T_LDV_C_': 'LD Car',
    'T_LDV_LT': 'LD Truck',
    'T_LDV_M': 'Motorcycle',
    'T_IMP_': 'Fuel use',
    'H2_distribution': 'Fuel use'
}

def map_tech_to_mode(tech):
    for prefix, class_name in class_mapping.items():
        if tech.startswith(prefix):
            return class_name
    return 'Other'

carrier_mapping = {
        'BEV': 'BEV',
        'GSL': 'Gasoline',
        'DSL': 'Diesel',
        'CNG': 'CNG',
        'LNG': 'LNG',
        'JTF': 'Jet Fuel',
        'SPK': 'Synthetic Jet Fuel', #      Might change for SAF
        'HFO': 'Heavy Fuel Oil',
        'MDO': 'Marine Diesel Oil',
        'H2': 'H2',
        'ELC': 'Electricity'
    }

def map_tech_to_fuel(tech):
    # Check hybrids first
    if 'PHEV35' in tech:
        return 'PHEV35'
    if 'PHEV50' in tech:
        return 'PHEV50'
    if 'PHEV' in tech:
        return 'PHEV'
    if 'BEV150' in tech:
        return 'BEV150'
    if 'BEV200' in tech:
        return 'BEV200'
    if 'BEV300' in tech:
        return 'BEV300'
    if 'BEV400' in tech:
        return 'BEV400'
    if 'FC' in tech:
        return 'FCEV'
    if 'HEV' in tech:
        return 'HEV'
    if 'RDSL' in tech:
            return 'Biodiesel' #            Might differentiate
    if 'ETH' in tech:
        return 'Ethanol' #                  Might differentiate
    for prefix, carrier in carrier_mapping.items():
        if prefix in tech:
            return carrier 
    return 'Other'

In [5]:
net_cap['mode'] = net_cap['tech'].apply(map_tech_to_mode)
new_cap['mode'] = new_cap['tech'].apply(map_tech_to_mode)
ex_cap['mode'] = ex_cap['tech'].apply(map_tech_to_mode)

net_cap['fuel'] = net_cap['tech'].apply(map_tech_to_fuel)
new_cap['fuel'] = new_cap['tech'].apply(map_tech_to_fuel)
ex_cap['fuel'] = ex_cap['tech'].apply(map_tech_to_fuel)

In [6]:
net_cap_group = net_cap.groupby(['mode', 'fuel', 'period'], as_index=False).sum('capacity').drop(columns='vintage').rename(columns={'period': 'vintage'})
net_cap_group[(net_cap_group['mode'] == 'LD Car') & (net_cap_group['fuel'] == 'Gasoline')].sort_values(by=['fuel', 'vintage'])

,mode,fuel,vintage,capacity
101,LD Car,Gasoline,2021,3601.864504
102,LD Car,Gasoline,2025,3015.336018
103,LD Car,Gasoline,2030,1658.204303
104,LD Car,Gasoline,2035,0.000987
105,LD Car,Gasoline,2040,0.000956
106,LD Car,Gasoline,2045,0.000976
107,LD Car,Gasoline,2050,0.001023


In [7]:
ex_cap_group = ex_cap.copy()
ex_cap_group['vintage'] = 2021
ex_cap_group = ex_cap_group.groupby(['mode', 'fuel', 'vintage', 'units'], as_index=False).sum('capacity')

In [8]:
new_cap_group = new_cap.groupby(['mode', 'fuel', 'vintage'], as_index=False).sum('capacity')
new_cap_group[(new_cap_group['mode'] == 'LD Car') & (new_cap_group['fuel'] == 'Gasoline')].sort_values(by=['vintage'])

,mode,fuel,vintage,capacity
88,LD Car,Gasoline,2021,0.000414
89,LD Car,Gasoline,2025,0.000365
90,LD Car,Gasoline,2030,0.000325
91,LD Car,Gasoline,2035,0.000297
92,LD Car,Gasoline,2040,0.000334
93,LD Car,Gasoline,2045,0.000346
94,LD Car,Gasoline,2050,0.000344


In [9]:
net_cap_group = net_cap_group[['mode', 'fuel', 'vintage', 'capacity']].rename(columns={'capacity': 'net capacity'})
new_cap_group = new_cap_group[['mode', 'fuel', 'vintage', 'capacity']].rename(columns={'capacity': 'new capacity'})
ex_cap_group = ex_cap_group[['mode', 'fuel', 'vintage', 'capacity']].rename(columns={'capacity': 'ex capacity'})

merged_cap = net_cap_group.merge(new_cap_group, on=['mode', 'fuel', 'vintage'], how='left').merge(ex_cap_group, on=['mode', 'fuel', 'vintage'], how='left').fillna(0.)

In [10]:
merged_cap[(merged_cap['mode'] == 'LD Car') & (merged_cap['fuel'] == 'Gasoline')].sort_values(by=['fuel', 'vintage'])

,mode,fuel,vintage,net capacity,new capacity,ex capacity
102,LD Car,Gasoline,2021,3601.864504,0.000414,3601.86409
103,LD Car,Gasoline,2025,3015.336018,0.000365,0.00000
104,LD Car,Gasoline,2030,1658.204303,0.000325,0.00000
105,LD Car,Gasoline,2035,0.000987,0.000297,0.00000
106,LD Car,Gasoline,2040,0.000956,0.000334,0.00000
107,LD Car,Gasoline,2045,0.000976,0.000346,0.00000
108,LD Car,Gasoline,2050,0.001023,0.000344,0.00000


In [11]:
merged_cap = merged_cap.sort_values(by=['mode', 'fuel', 'vintage'])
merged_cap['retired cap'] = 0.

for _, group in merged_cap.groupby(['mode', 'fuel']):
    for i in range(1, len(group)):
        current_idx = group.index[i]
        previous_idx = group.index[i-1]

        # Calculate retired capacity as existing capacity + new capacity - net capacity of previous vintage
        merged_cap.loc[current_idx, 'retired cap'] = (
            merged_cap.loc[previous_idx, 'net capacity'] + 
            merged_cap.loc[current_idx, 'new capacity'] - 
            merged_cap.loc[current_idx, 'net capacity']
        )

for _, group in merged_cap.groupby(['mode', 'fuel']):
    for i in range(1, len(group)):
        current_idx = group.index[i]
        previous_idx = group.index[i-1]

        # Fill the "ex capacity" for years after 2021 using the "net capacity" from the previous year
        merged_cap.loc[current_idx, 'ex capacity'] = merged_cap.loc[previous_idx, 'net capacity']

merged_cap['retired cap'] = -merged_cap['retired cap']

In [12]:
merged_cap[(merged_cap['mode'] == 'LD Car') & (merged_cap['fuel'] == 'BEV150')]

,mode,fuel,vintage,net capacity,new capacity,ex capacity,retired cap
53,LD Car,BEV150,2021,496.766162,455.949252,40.816910,-0.000000
54,LD Car,BEV150,2025,2481.119297,1984.353135,496.766162,-0.000000
55,LD Car,BEV150,2030,3969.402850,1491.919673,2481.119297,-3.636120
56,LD Car,BEV150,2035,6157.373846,2681.101039,3969.402850,-493.130042
57,LD Car,BEV150,2040,6473.889380,2300.868669,6157.373846,-1984.353135
58,LD Car,BEV150,2045,7120.629901,2138.660193,6473.889380,-1491.919673
59,LD Car,BEV150,2050,7510.732132,3071.203270,7120.629901,-2681.101039


In [13]:
merged_cap[(merged_cap['mode'] == 'LD Car') & (merged_cap['fuel'] == 'Gasoline')]

,mode,fuel,vintage,net capacity,new capacity,ex capacity,retired cap
102,LD Car,Gasoline,2021,3601.864504,0.000414,3601.864090,-0.000000
103,LD Car,Gasoline,2025,3015.336018,0.000365,3601.864504,-586.528850
104,LD Car,Gasoline,2030,1658.204303,0.000325,3015.336018,-1357.132040
105,LD Car,Gasoline,2035,0.000987,0.000297,1658.204303,-1658.203614
106,LD Car,Gasoline,2040,0.000956,0.000334,0.000987,-0.000365
107,LD Car,Gasoline,2045,0.000976,0.000346,0.000956,-0.000325
108,LD Car,Gasoline,2050,0.001023,0.000344,0.000976,-0.000297


In [14]:
merged_cap[merged_cap['mode'] == 'LD Car'].groupby(['mode', 'vintage']).sum(['net capacity', 'new capacity', 'ex capacity', 'retired cap'])

net capacity  new capacity  ex capacity  retired cap
mode   vintage                                                      
LD Car 2021      4271.480563    455.954883  3815.525680     0.000000
       2025      5644.066769   1984.358126  4271.480563  -611.771920
       2030      5701.785259   1491.925660  5644.066769 -1434.207170
       2035      6157.391406   2681.107619  5701.785259 -2225.501473
       2040      6473.909866   2300.876587  6157.391406 -1984.358126
       2045      7120.651834   2138.667628  6473.909866 -1491.925660
       2050      7510.755818   3071.211603  7120.651834 -2681.107619

In [15]:
cap = merged_cap.melt(id_vars=['mode', 'fuel', 'vintage'],
                       value_vars=['net capacity', 'ex capacity', 'new capacity', 'retired cap'],
                       var_name='cap type', value_name='capacity')

cap['capacity'] = cap['capacity'] / 1E3
cap = cap[(abs(cap['capacity']) > 1e-3) & (
                                            (cap['mode'] == 'MD Truck') |
                                            (cap['mode'] == 'HD Truck')
                                            ) & (cap['cap type'] != 'ex capacity')].reset_index(drop=True)
cap['vintage'] = cap['vintage'].astype('str')

# cap.to_csv('car_cap.csv', index=False)
# cap.to_clipboard()

In [16]:
fig = px.bar(cap, x='cap type', y='capacity', color='fuel', 
            pattern_shape='cap type', pattern_shape_sequence=["", ".", "/"],
            facet_col='vintage', facet_col_spacing=2E-2, 
            facet_row='mode', facet_row_spacing=1E-2,
            category_orders={"cap type": ["net capacity", "new capacity", "retired cap"],
                             "fuel": ['Gasoline', 'Diesel', 'CNG', 'BEV150', 'BEV200', 'BEV300', 'BEV400', 'HEV', 'PHEV35', 'PHEV50', 'FCEV']},
            # labels={'capacity': 'Fleet capacity (M units)', 'fuel': 'Fuel type'},
            template='plotly_white', orientation='v', color_discrete_sequence=px.colors.qualitative.Bold,
            text_auto='.2f',
            width=1200, height=800
            )

dummy_traces = [
    dict(
        name='',  # Empty legend
        x=[None], 
        y=[None], 
        marker=dict(color='rgba(0,0,0,0)'),
        showlegend=True,
        legendgroup='Blank',
        legendgrouptitle=None
    ),
    # dict(
    #     name=' ',  # Another empty legend
    #     x=[None], 
    #     y=[None], 
    #     marker=dict(color='rgba(0,0,0,0)'),
    #     showlegend=True,
    #     legendgroup='Blank',
    #     legendgrouptitle=None   
    # ),
    dict(
        name='Net capacity',
        x=[None], 
        y=[None], 
        marker=dict(
            color='rgba(0,0,0,0)',
            line=dict(color='black', width=1),  # Thin black border
            pattern_shape=""
        ),
        showlegend=True,
        legendgroup='Capacity type',
        legendgrouptitle=dict(text="Capacity type"),
    ),
    dict(
        name='New capacity',
        x=[None], 
        y=[None], 
        marker=dict(
            color='rgba(0,0,0,0)',
            line=dict(color='black', width=1),  # Thin black border
            pattern_shape="."
        ),
        showlegend=True,
        legendgroup='Capacity type',
        legendgrouptitle=None
    ),
    dict(
        name='Retired capacity',
        x=[None], 
        y=[None], 
        marker=dict(
            color='rgba(0,0,0,0)',
            line=dict(color='black', width=1),  # Thin black border
            pattern_shape="/"
        ),
        showlegend=True,
        legendgroup='Capacity type',
        legendgrouptitle=None
    )
]

for trace in dummy_traces:
    fig.add_bar(
        x=trace['x'], 
        y=trace['y'], 
        name=trace['name'], 
        marker=trace['marker'], 
        showlegend=trace['showlegend'],
        legendgroup=trace['legendgroup'],
        legendgrouptitle=trace['legendgrouptitle']
    )

fig.update_layout(
    margin=dict(
        t=10, b=10
    ),
    title=dict(
        text='Transport LDV fleet stock and flow in ON by vehicle class and drivetrain',
        x=0.5, 
        y=0.97,
        xanchor='center',
        yanchor='top'
    ),
    yaxis_title_standoff=0,
    legend_title_text='Fuel type',
    bargap=0.1,
    legend=dict(
        orientation='v',
        # entrywidth=50,
        yanchor='top',
        y=0.8,  
        xanchor='center',
        x=1.15),
    font=dict(
        size=15)
    )

fig.for_each_trace(lambda trace: trace.update(textfont=dict(size=11)))

# fig.update_xaxes(tickvals=["net capacity", "new capacity", "retired cap"],
#                  ticktext=["Net", "New", "Retired"], 
#                  tickangle=45
#                  )

for axis in fig.layout:
    if axis.startswith('xaxis'):
        fig.layout[axis].title.text = ''
        fig.layout[axis].showticklabels = False
    if axis.startswith('yaxis'):
        fig.layout[axis].title.text = ''
        fig.layout[axis].dtick = 2

for annotation in fig.layout.annotations:
    if 'mode' in annotation.text:
        annotation.text = 'Class: ' + annotation.text.split('=')[1]
        annotation.font.size = 16
        annotation.x = 1.01
        annotation.xanchor = 'center'
    else:
        annotation.text = annotation.text.split('=')[1]
        annotation.y = 0.005
        annotation.yanchor = 'top' 

shown_legends = set()
for trace in fig.data:
    trace.name = trace.name.split(",")[0]
    if trace.name not in shown_legends:
        trace.showlegend = True
        shown_legends.add(trace.name)
    else:
        trace.showlegend = False

fig.add_annotation(
    text="Fleet capacity (M units)",
    x=-0.07,
    y=0.5,
    xref="paper",
    yref="paper",
    showarrow=False,
    textangle=-90,
    font=dict(size=17))

fig.show()

# fig.write_image("vanilla2_fleet_flow.svg", scale=1, engine='kaleido')

In [17]:
# fig = px.bar(cap[cap['cap type'] != 'net capacity'], x='vintage', y='capacity', color='fuel', pattern_shape='cap type', 
#             #  facet_col='vintage', facet_col_spacing=2E-2,
#              pattern_shape_sequence=["", ".", "/"],
#              labels={'vintage':'Period', 'capacity':'Capacity (M units)'},
#              template='plotly_white', orientation='v', color_discrete_sequence=px.colors.qualitative.Bold,
#              text_auto='.3f',
#             #  category_orders={"scenario": ['CP', 'CP TM', 'NZ', 'NZ TM']},
#             width=1100, height=750
#             )

# total_cap = cap[cap['cap type'] == 'net capacity'].groupby('vintage', as_index=False).sum('capacity')
# fig.add_trace(go.Scatter(x=total_cap['vintage'], y=total_cap['capacity'],
#                          name='Net capacity', mode='lines+markers',
#                          line=dict(color='black', width=2),
#                          marker=dict(size=6)
#                          ))

# fig.update_layout(title=dict(
#         text='Transport LDV fleet stock and flow in ON by vehicle class and drivetrain',
#         x=0.5, 
#         y=0.97,
#         xanchor='center',
#         yanchor='top'
#     ),
#     # xaxis=dict(title='Period'),
#     yaxis=dict(
#         # title='Capacity (k units)',
#             #    dtick=1
#                ),
#     yaxis_title_standoff=0,
#     legend_title_text='Fuel type',
#     bargap=0.25,
#     # legend=dict(
#     #     orientation='h',  
#     #     yanchor='top',
#     #     y=1.03,  
#     #     xanchor='center',
#     #     x=0.5),
#     font=dict(
#         # family="Open Sans",
#         size=14)
#     )

# for annotation in fig.layout.annotations:
#     annotation.text = annotation.text.split('=')[1]
#     annotation.y = -0.08  
#     annotation.yanchor = 'top' 

# # for axis in fig.layout:
# #     if axis.startswith('yaxis'):
# #         fig.layout[axis].dtick = 1

# # for axis in fig.layout:
# #     if axis.startswith('xaxis'):
# #         fig.layout[axis].title.text = ''

# fig.for_each_trace(lambda trace: trace.update(textfont=dict(size=9)))

# fig.show()

In [18]:
# periods = ld_cap['vintage'].unique()
# fuels = ld_cap['fuel'].unique()

# # Prepare positions for the bars and assign colors for each fuel type
# bar_width = 0.25
# positions = np.arange(len(periods))
# colors = plt.cm.get_cmap('tab20', len(fuels))

# # Sorting and calculating cumulative sums separately for each capacity flow type (existing, retired, new)
# cumulative_data = pd.DataFrame()

# for vintage in periods:
#     # Filter data for the current vintage
#     vintage_data = ld_cap[ld_cap['vintage'] == vintage].copy()
    
#     # Ensure all fuels are present, reindex to match all fuel types
#     vintage_data = vintage_data.set_index('fuel').reindex(fuels).fillna(0)
    
#     # Reset index back to fuel
#     vintage_data = vintage_data.reset_index()
    
#     # Sort each capacity flow type (existing, retired, and new) in descending order separately
#     vintage_data['ex capacity'] = vintage_data['ex capacity'].sort_values(ascending=False).values
#     vintage_data['cumulative_ex_capacity'] = vintage_data['ex capacity'].cumsum()

#     vintage_data['retired cap'] = vintage_data['retired cap'].sort_values(ascending=False).values
#     vintage_data['cumulative_retired_capacity'] = vintage_data['retired cap'].cumsum()

#     vintage_data['new capacity'] = vintage_data['new capacity'].sort_values(ascending=False).values
#     vintage_data['cumulative_new_capacity'] = vintage_data['new capacity'].cumsum()
    
#     # Append to the cumulative dataframe
#     cumulative_data = pd.concat([cumulative_data, vintage_data])

# # Plot the waterfall chart
# fig, ax = plt.subplots(figsize=(12, 7))

# for i, fuel in enumerate(fuels):
#     fuel_data = cumulative_data[cumulative_data['fuel'] == fuel]
    
#     # Plot existing capacity (stacked cumulatively)
#     ax.bar(positions - bar_width - 0.05, fuel_data['cumulative_ex_capacity'], bar_width, label=f'{fuel}', color=colors(i))
    
#     # Plot retired capacity (negative and cumulative)
#     retired_position = fuel_data['cumulative_ex_capacity']
#     ax.bar(positions, -fuel_data['cumulative_retired_capacity'], bar_width, bottom=retired_position, color=colors(i))
    
#     # Plot new capacity (cumulative)
#     new_position = retired_position - fuel_data['cumulative_retired_capacity']
#     ax.bar(positions + bar_width + 0.05, fuel_data['cumulative_new_capacity'], bar_width, bottom=new_position, color=colors(i))

# # Chart formatting
# ax.set_title('Car Fleet Capacity (Cumulative) by Vintage and Fuel Type')
# ax.set_xlabel('Vintage (Year)')
# ax.set_ylabel('Cumulative Capacity')
# ax.set_xticks(positions)
# ax.set_xticklabels(periods)
# ax.legend(loc='upper left', bbox_to_anchor=(1, 1), title="Fuel Type")
# # plt.grid(True)
# plt.tight_layout()

# # Show the final cumulative waterfall chart
# plt.show()

In [19]:
# fig = go.Figure()

# # Handle the two stackgroups: positive for 'ex capacity' + 'new capacity' and negative for 'retired cap'
# for fuel_type in car_cap['fuel'].unique():
#     # Separate positive and negative capacities
#     df_fuel_pos = car_cap[(car_cap['fuel'] == fuel_type) & (car_cap['capacity'] > 0)]
#     df_fuel_neg = car_cap[(car_cap['fuel'] == fuel_type) & (car_cap['capacity'] < 0)]

#     # Add positive capacities to the 'positive' stackgroup
#     fig.add_trace(go.Scatter(
#         x=df_fuel_pos['vintage'],
#         y=df_fuel_pos['capacity'],
#         mode='lines',
#         name=f'{fuel_type} (positive)',
#         stackgroup='positive',  # Positive stackgroup
#         line=dict(width=0.5),
#         fill='tonexty'
#     ))

#     # Add negative capacities to the 'negative' stackgroup
#     fig.add_trace(go.Scatter(
#         x=df_fuel_neg['vintage'],
#         y=df_fuel_neg['capacity'],
#         mode='lines',
#         name=f'{fuel_type} (negative)',
#         stackgroup='negative',  # Negative stackgroup
#         line=dict(width=0.5),
#         fill='tonexty'
#     ))

# # Update the layout for proper visualization
# fig.update_layout(
#     title="Stacked Capacity by Fuel and Vintage (Positive and Negative Values)",
#     xaxis_title="Vintage",
#     yaxis_title="Capacity",
#     yaxis_rangemode="normal",  # Ensures negative values are displayed correctly
#     showlegend=True,
#     barmode='group'  # Ensures positive and negative stackgroups are separate
# )

# fig.update_xaxes(showgrid=False)
# fig.update_layout(width=800,
#                   height=600,
#                   # title={'x':0.5,
#                   #        'y':0.88},
#                   # legend=dict(orientation='v',
#                   #             y=1,
#                   #             x=0.8)
#                  )

# fig.show()

In [20]:
# pivoted_data = car_cap.pivot_table(index=['vintage', 'fuel', 'cap type'], values='capacity', aggfunc='sum').fillna(0).reset_index()

# # Initialize a figure object
# fig = go.Figure()

# # Define a fillpattern for each capacity type
# fill_patterns = {
#     'ex capacity': None,  # No pattern for existing capacity
#     'new capacity': {'shape': '/'},  # Diagonal stripes for new capacity
#     'retired cap': {'shape': '.'}   # Dots for retired capacity
# }

# # Separate stackgroups for positive and negative values
# for fuel in pivoted_data['fuel'].unique():
#     # Loop for positive values (new and ex capacity)
#     for cap_type, pattern in fill_patterns.items():
#         if cap_type != 'retired cap':  # Only positive stackgroup for non-retired capacities
#             fuel_data = pivoted_data[(pivoted_data['fuel'] == fuel) & (pivoted_data['cap type'] == cap_type)]
#             fig.add_trace(go.Scatter(
#                 x=fuel_data['vintage'],
#                 y=fuel_data['capacity'],
#                 stackgroup='positive',  # Positive stackgroup
#                 name=f"{fuel}",
#                 showlegend=(cap_type == 'ex capacity'),  # Show legend only once per fuel
#                 fill='tonexty',
#                 fillpattern=pattern if pattern else None,
#                 hoverinfo='x+y+name'
#             ))
    
#     # Loop for negative values (retired capacity)
#     retired_data = pivoted_data[(pivoted_data['fuel'] == fuel) & (pivoted_data['cap type'] == 'retired cap')]
#     fig.add_trace(go.Scatter(
#         x=retired_data['vintage'],
#         y=retired_data['capacity'],  # Negative values handled separately
#         stackgroup='negative',  # Negative stackgroup for retired capacities
#         name=f"{fuel}",
#         fill='tonexty',
#         fillpattern=fill_patterns['retired cap'],
#         hoverinfo='x+y+name',
#         showlegend=False  # Legend already added for the fuel
#     ))

# # Add a custom legend for the capacity type patterns using empty filled areas
# pattern_legend_items = [
#     go.Scatter(
#         x=[None], y=[None],
#         mode='lines',
#         fill='toself',
#         fillpattern=fill_patterns[cap_type],
#         name=cap_type
#     ) for cap_type in fill_patterns.keys() if fill_patterns[cap_type]  # Only create a legend item if there's a pattern
# ]

# # Add pattern legend items to the figure
# for item in pattern_legend_items:
#     fig.add_trace(item)

# # Customize the layout
# fig.update_layout(
#     title="Stacked Area Plot with Patterns for Capacity Types",
#     xaxis_title="Vintage",
#     yaxis_title="Capacity",
#     showlegend=True,
#     legend=dict(title="Fuel Type and Capacity Type"),
#     barmode='group',  # Grouped mode for the bar stacks
#     yaxis=dict(rangemode="normal")  # Ensure y-axis includes negative values
# )

# # Show or save the plot
# # For displaying:
# fig.show()

In [21]:
def map_tech_survival(tech):
    if '_S25' in tech:
        return 'Survival Q1'
    if '_S75' in tech:
        return 'Survival Q4'
    else: return 'Survival Q2-3'

if 'lifetime' in db_name:
    net_cap['life'] = net_cap['tech'].apply(map_tech_survival)
    new_cap['life'] = new_cap['tech'].apply(map_tech_survival)
    ex_cap['life'] = ex_cap['tech'].apply(map_tech_survival)
    net_cap_group = net_cap.groupby(['mode', 'fuel', 'life', 'period'], as_index=False).sum('capacity').drop(columns='vintage').rename(columns={'period': 'vintage'})
    ex_cap_group = ex_cap.copy()
    ex_cap_group['vintage'] = 2021
    ex_cap_group = ex_cap_group.groupby(['mode', 'fuel', 'life', 'vintage', 'units'], as_index=False).sum('capacity')
    new_cap_group = new_cap.groupby(['mode', 'fuel', 'life', 'vintage'], as_index=False).sum('capacity')
    net_cap_group = net_cap_group[['mode', 'fuel', 'life', 'vintage', 'capacity']].rename(columns={'capacity': 'net capacity'})
    new_cap_group = new_cap_group[['mode', 'fuel', 'life', 'vintage', 'capacity']].rename(columns={'capacity': 'new capacity'})
    ex_cap_group = ex_cap_group[['mode', 'fuel', 'life', 'vintage', 'capacity']].rename(columns={'capacity': 'ex capacity'})

    merged_cap = net_cap_group.merge(new_cap_group, on=['mode', 'fuel', 'life', 'vintage'], how='left').merge(ex_cap_group, on=['mode', 'fuel', 'life', 'vintage'], how='left').fillna(0.)
    merged_cap = merged_cap.sort_values(by=['mode', 'fuel', 'life', 'vintage'])
    merged_cap['retired cap'] = 0.

    for _, group in merged_cap.groupby(['mode', 'fuel', 'life']):
        for i in range(1, len(group)):
            current_idx = group.index[i]
            previous_idx = group.index[i-1]

            # Calculate retired capacity as existing capacity + new capacity - net capacity of previous vintage
            merged_cap.loc[current_idx, 'retired cap'] = (
                merged_cap.loc[previous_idx, 'net capacity'] + 
                merged_cap.loc[current_idx, 'new capacity'] - 
                merged_cap.loc[current_idx, 'net capacity']
            )

    for _, group in merged_cap.groupby(['mode', 'fuel', 'life']):
        for i in range(1, len(group)):
            current_idx = group.index[i]
            previous_idx = group.index[i-1]

            # Fill the "ex capacity" for years after 2021 using the "net capacity" from the previous year
            merged_cap.loc[current_idx, 'ex capacity'] = merged_cap.loc[previous_idx, 'net capacity']

    merged_cap['retired cap'] = -merged_cap['retired cap']

    cap = merged_cap.melt(id_vars=['mode', 'fuel', 'life', 'vintage'],
                       value_vars=['net capacity', 'ex capacity', 'new capacity', 'retired cap'],
                       var_name='cap type', value_name='capacity')

    cap['capacity'] = cap['capacity'] / 1E3
    cap = cap[(abs(cap['capacity']) > 1e-3) & (
                                                (cap['mode'] == 'LD Car') |
                                                (cap['mode'] == 'LD Truck')
                                                ) & (cap['cap type'] != 'ex capacity')].reset_index(drop=True)
    cap['vintage'] = cap['vintage'].astype('str')

cap = cap.groupby(['mode', 'life', 'vintage', 'cap type'], as_index=False).sum('capacity')
fig = px.bar(cap, x='cap type', y='capacity', color='life', 
            pattern_shape='cap type', pattern_shape_sequence=["", ".", "/"],
            facet_col='vintage', facet_col_spacing=2E-2, 
            facet_row='mode', facet_row_spacing=1E-2,
            category_orders={"cap type": ["net capacity", "new capacity", "retired cap"],
                             "fuel": ['Gasoline', 'Diesel', 'CNG', 'BEV200', 'BEV300', 'BEV400', 'BEV500', 'HEV', 'PHEV20', 'PHEV50', 'FCEV'],
                             "life": ['Survival Q1', 'Survival Q2-3', 'Survival Q4']},
            # labels={'capacity': 'Fleet capacity (M units)', 'fuel': 'Fuel type'},
            template='plotly_white', orientation='v', color_discrete_sequence=px.colors.qualitative.Bold,
            text_auto='.2f',
            width=1200, height=800
            )

dummy_traces = [
    dict(
        name='',  # Empty legend
        x=[None], 
        y=[None], 
        marker=dict(color='rgba(0,0,0,0)'),
        showlegend=True,
        legendgroup='Blank',
        legendgrouptitle=None
    ),
    # dict(
    #     name=' ',  # Another empty legend
    #     x=[None], 
    #     y=[None], 
    #     marker=dict(color='rgba(0,0,0,0)'),
    #     showlegend=True,
    #     legendgroup='Blank',
    #     legendgrouptitle=None   
    # ),
    dict(
        name='Net capacity',
        x=[None], 
        y=[None], 
        marker=dict(
            color='rgba(0,0,0,0)',
            line=dict(color='black', width=1),  # Thin black border
            pattern_shape=""
        ),
        showlegend=True,
        legendgroup='Capacity type',
        legendgrouptitle=dict(text="Capacity type"),
    ),
    dict(
        name='New capacity',
        x=[None], 
        y=[None], 
        marker=dict(
            color='rgba(0,0,0,0)',
            line=dict(color='black', width=1),  # Thin black border
            pattern_shape="."
        ),
        showlegend=True,
        legendgroup='Capacity type',
        legendgrouptitle=None
    ),
    dict(
        name='Retired capacity',
        x=[None], 
        y=[None], 
        marker=dict(
            color='rgba(0,0,0,0)',
            line=dict(color='black', width=1),  # Thin black border
            pattern_shape="/"
        ),
        showlegend=True,
        legendgroup='Capacity type',
        legendgrouptitle=None
    )
]

for trace in dummy_traces:
    fig.add_bar(
        x=trace['x'], 
        y=trace['y'], 
        name=trace['name'], 
        marker=trace['marker'], 
        showlegend=trace['showlegend'],
        legendgroup=trace['legendgroup'],
        legendgrouptitle=trace['legendgrouptitle']
    )

fig.update_layout(
    margin=dict(
        t=10, b=10
    ),
    title=dict(
        text='Transport LDV fleet stock and flow in ON by vehicle class and lifetime',
        x=0.5, 
        y=0.97,
        xanchor='center',
        yanchor='top'
    ),
    yaxis_title_standoff=0,
    legend_title_text='Survival rate class',
    bargap=0.1,
    legend=dict(
        orientation='v',
        # entrywidth=50,
        yanchor='top',
        y=0.8,  
        xanchor='center',
        x=1.15),
    font=dict(
        size=15)
    )

fig.for_each_trace(lambda trace: trace.update(textfont=dict(size=11)))

# fig.update_xaxes(tickvals=["net capacity", "new capacity", "retired cap"],
#                  ticktext=["Net", "New", "Retired"], 
#                  tickangle=45
#                  )

for axis in fig.layout:
    if axis.startswith('xaxis'):
        fig.layout[axis].title.text = ''
        fig.layout[axis].showticklabels = False
    if axis.startswith('yaxis'):
        fig.layout[axis].title.text = ''
        fig.layout[axis].dtick = 2

for annotation in fig.layout.annotations:
    if 'mode' in annotation.text:
        annotation.text = 'Class: ' + annotation.text.split('=')[1]
        annotation.font.size = 16
        annotation.x = 1.01
        annotation.xanchor = 'center'
    else:
        annotation.text = annotation.text.split('=')[1]
        annotation.y = 0.005
        annotation.yanchor = 'top' 

shown_legends = set()
for trace in fig.data:
    trace.name = trace.name.split(",")[0]
    if trace.name not in shown_legends:
        trace.showlegend = True
        shown_legends.add(trace.name)
    else:
        trace.showlegend = False

fig.add_annotation(
    text="Fleet capacity (M units)",
    x=-0.07,
    y=0.5,
    xref="paper",
    yref="paper",
    showarrow=False,
    textangle=-90,
    font=dict(size=17))

fig.show()

fig.write_image("lifetimes_fleet_flow.svg", scale=1, engine='kaleido')

KeyError: 'life'